In [135]:
from tensorflow.keras.layers import Dense, Conv2D, GlobalAveragePooling2D,BatchNormalization,Activation, Add,Flatten,ZeroPadding2D,MaxPooling2D,AveragePooling2D
from tensorflow.keras import activations, Input, Model
from tensorflow.keras.regularizers import l2
import tensorflow as tf
import numpy as np

In [136]:
def res_block(x,filters,down_sampling=False): 
    x_prev = x
    if down_sampling == False:
        x = Conv2D(filters,kernel_size=(3, 3), strides=(1, 1),padding='same',kernel_initializer='he_normal'
                   , kernel_regularizer=l2(1e-4))(x)
    else:
        x = Conv2D(filters,kernel_size=(3, 3), strides=(2, 2),padding='same',kernel_initializer='he_normal'
           ,kernel_regularizer=l2(1e-4))(x)
    x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)
    
    x = Conv2D(filters, kernel_size=(3, 3), strides=(1, 1),padding='same',kernel_initializer='he_normal'
              ,kernel_regularizer=l2(1e-4))(x)
    x = BatchNormalization()(x)
    
    if down_sampling == False:
        x = Add()([x, x_prev])
        x = Activation(activations.relu)(x)
    else:
        x_prev = AveragePooling2D(pool_size=(2, 2), strides=(2,2), padding='same')(x_prev)
        x_prev = BatchNormalization()(x_prev)
        pad = int(filters/4)
        x_prev = tf.pad(x_prev,[[0,0],[0,0],[0,0],[pad,pad]])
        x = Add()([x, x_prev])
        x = Activation(activations.relu)(x)
    return x

In [137]:
def resnet110(input_shape,num_class):

    input_im = Input(shape=input_shape) # cifar 10 images size

    x = Conv2D(16, kernel_size=(3, 3),kernel_initializer='he_normal',padding='same'
              ,kernel_regularizer=l2(1e-4))(input_im)
    x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)

    for i in range(9):
        x = res_block(x, filters=16)
    
    x = res_block(x, filters=32,down_sampling=True)
    for i in range(8):
        x = res_block(x, filters=32)
    
    x = res_block(x, filters=64,down_sampling=True)
    for i in range(8):
        x = res_block(x, filters=64)



    x = GlobalAveragePooling2D()(x)

    x = Dense(num_class, activation='softmax', kernel_initializer='he_normal'
             ,kernel_regularizer=l2(1e-4))(x) #multi-class

    # define the model 

    model = Model(inputs=input_im, outputs=x, name='Resnet110')

    return model

In [143]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train = (x_train - np.mean(x_train,axis=0))/np.std(x_train,axis=0)
x_test = (x_test - np.mean(x_test,axis=0))/np.std(x_test,axis=0)
np.random.seed(42)
mask_val = np.random.choice(50000,5000,replace=False)
mask_train = np.array([i for i in range(50000) if i not in mask_val])
x_val, y_val = x_train[mask_val], y_train[mask_val]
x_train, y_train = x_train[mask_train], y_train[mask_train]
print(x_train.shape)
print(x_test.shape)

In [144]:
input_shape = x_train.shape[1:]
model = resnet110(input_shape,10)

(45000, 32, 32, 3)

In [145]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1,nesterov=True)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
batch_size = 128
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(batch_size)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
def pad_crop(image,pad=4,crop_h=32,crop_w=32):
    '''
    Input should be a 3D tensor, (height, width, channels)
    Output is a processed 3D tensor, (crop_h, crop_w, channels)
    custom function for ImageDataGenerator.
    First padd the image then randomly crop.
    '''
    height, width = image.shape[0], image.shape[1]
    padded = tf.image.pad_to_bounding_box(image,pad,pad,height+2*pad,width+2*pad)
    crop = tf.image.random_crop(padded,size=(crop_h,crop_w,3), seed=42)
    return crop
img_datagen = ImageDataGenerator(horizontal_flip=True,preprocessing_function = pad_crop)
itr = img_datagen.flow(x_train,y_train,batch_size=batch_size)

Model: "Resnet110"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_119 (Conv2D)             (None, 32, 32, 16)   448         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_113 (BatchN (None, 32, 32, 16)   64          conv2d_119[0][0]                 
__________________________________________________________________________________________________
activation_165 (Activation)     (None, 32, 32, 16)   0           batch_normalization_113[0][0]    
__________________________________________________________________________________________

In [149]:
import time
epochs = 100
history = [[],[],[]]
for epoch in range(epochs):
    print("Epoch %d/%d" % (epoch+1,epochs))
    if (epoch==50) | (epoch==75):
        optimizer.learning_rate = optimizer.learning_rate/10
    
    start_time = time.time()
    step = 0
    for x_batch_train, y_batch_train in itr:
        with tf.GradientTape() as tape:
            logits = model(x_batch_train, training=True)
            loss_value = loss_fn(y_batch_train, logits)
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        train_acc_metric.update_state(y_batch_train, logits)
        if step % 25 == 0:
            print(
                "Training loss at step %d: %.4f"
                % (step, float(loss_value))
            )
        step += 1
        if step > len(x_train)/batch_size:
            break

    history[0].append(loss_value)
    train_acc = train_acc_metric.result()
    train_acc_metric.reset_states()

    for x_batch_val, y_batch_val in val_dataset:
        val_logits = model(x_batch_val, training=False)
        val_acc_metric.update_state(y_batch_val, val_logits)
    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()
    history[1].append(train_acc)
    history[2].append(val_acc)
    print("Training accuracy: %.4f" % (float(train_acc),)
          ,"Validation accuracy: %.4f" % (float(val_acc),),"Time taken: %.2fs" % (time.time() - start_time))


In [150]:
batch_size = 512
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(batch_size)

In [151]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
def pad_crop(image,pad=4,crop_h=32,crop_w=32):
    height, width = image.shape[0], image.shape[1]
    padded = tf.image.pad_to_bounding_box(image,pad,pad,height+2*pad,width+2*pad)
    crop = tf.image.random_crop(padded,size=(crop_h,crop_w,3), seed=42)
    return crop
img_datagen = ImageDataGenerator(horizontal_flip=True,preprocessing_function = pad_crop)
itr = img_datagen.flow(x_train,y_train,batch_size=batch_size)

In [152]:
import time
epochs = 50
history = [[],[],[]]
for epoch in range(epochs):
    print("Epoch %d/%d" % (epoch+1,epochs))
    
    start_time = time.time()
    step = 0
    for x_batch_train, y_batch_train in itr:
        with tf.GradientTape() as tape:
            logits = model(x_batch_train, training=True)
            loss_value = loss_fn(y_batch_train, logits)
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        train_acc_metric.update_state(y_batch_train, logits)
        if step % 25 == 0:
            print(
                "Training loss at step %d: %.4f"
                % (step, float(loss_value))
            )
        step += 1
        if step >= len(x_train)/batch_size:
            break

    history[0].append(loss_value)
    train_acc = train_acc_metric.result()
    train_acc_metric.reset_states()

    for x_batch_val, y_batch_val in val_dataset:
        val_logits = model(x_batch_val, training=False)
        val_acc_metric.update_state(y_batch_val, val_logits)
    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()
    history[1].append(train_acc)
    history[2].append(val_acc)
    print("Training accuracy: %.4f" % (float(train_acc),)
          ,"Validation accuracy: %.4f" % (float(val_acc),),"Time taken: %.2fs" % (time.time() - start_time))


Epoch 1/50
Training loss at step 0: 9.5316
Training loss at step 25: 3.6230


KeyboardInterrupt: 

In [125]:
conv = Conv2D(filters=16,kernel_size=(1,1),strides=(2,2),)

In [126]:
conv.trainable = False

In [129]:
x = x_train[1:2]

In [134]:
y = conv(x.astype('float32'))